![Py4Eng](img/logo.png)

# Graphical user interface
## Yoav Ram

# *Qt* GUI

We will be using [*Qt*](http://qt.io/), a cross-platform native application framework. There are two Python binding for the *Qt GUI toolkit*: *PyQt* and *PySide*; we will use the latter. Qt plays nicely with *Matplotlib* and with the notebook.

We'll need to  install *Qt* with `conda install qt` and then *PySide* with `pip install pyside` (*conda* currently doesn't have an installer for Python 3.4 and 3.5).

## Hello World!

Let's start with a simple *Hello World!* application - just a window with a button; when the button is clicked, an *Hello World!* dialog comes up. This example follows [Learning IPython for Interactive Computing and Data Visualization](http://ipython-books.github.io/minibook/) by Cyrille Rossant, pg. 85.

We use the notebook's magic command `%gui` to let the notebook know that we are using *Qt*, then import the `QtGui` module from `PySide`

In [12]:
%gui qt
from PySide import QtGui
import PySide
print("PySide version: ", PySide.__version__)

PySide version:  1.2.2


Next, we define out main application window, a class we call `HelloWorld`, which inherits from `QtGui.QWidget`. We add a push-button, with the label `Click me`, and connect it to the method `clicked`. 

We then create a simple layout and show the window (since it's the main window of the application). 

The `clicked` method creates a dialog with an `OK` button (called a  `QMessageBox`) which says `Hello World!`.

Finally, we create the window (it will show itself because we called `self.show()` in its `__init__`.

In [14]:
class HelloWorld(QtGui.QWidget):
    def __init__(self):
        super().__init__()
        self.button = QtGui.QPushButton('Click me', self)
        self.button.clicked.connect(self.clicked)
        # create the layout
        vbox = QtGui.QVBoxLayout()
        vbox.addWidget(self.button)
        self.setLayout(vbox)
        # show the window
        self.show()
    
    def clicked(self):
        msg = QtGui.QMessageBox(self)
        msg.setText("Hello World!")
        msg.show()

window = HelloWorld()

Interestingly, starting the GUI **doesn't block** the notebook (you can see the empty rather than filled circle at the top right of the notebook) which means we can interact with our window through the notebook. This is very useful for testing and debugging.

For example, we can trigger the `clicked` method without actually clicking the button:

In [3]:
window.clicked()

Change the window title and size:

In [4]:
window.setWindowTitle("Main Window")
window.resize(500, 50)

And close the window:

In [5]:
window.close()

True

## Qt Designer

When creating more sophisticated application, it's more convinient to work with a designer - a WYSIWYG GUI editor. The *Qt Creator* IDE has such functionality, allowing us to create and edit *.ui* files that define the layout and design of a *Qt* GUI application. We then translate this *.ui* file to a *.py* file using the *pyside-uic* tool. We then import the generated *.py* file, connect methods (callbacks) and run the application. 

Let's do a simple example before diving into a more sophisticated example. Open the *Qt Creator* application on your desktop. Click on the *File* menu, choose *New File or Project*, select *Qt* from the *Files and Classes* list, select the *Qt Designer Form* option, and click *Choose...*. Select *Main Window*, and click *Next*. Now choose a filename and path for the *.ui* file - remember this path as we will need it for converting to *.py* with *pyside-uic*. 

At this point, the *Qt Designer* will open and we can start adding widgets to it and design our GUI:

![QtDesigner](img/QtDesigner.png)

We'll build a simple app with just a big textbox to write text to and save\load to\from a file.

The design is implemented in `../scripts/notepad.ui` - you can open it in the *Qt Designer* (File -> Open File or Project).

![QtDesigner Notepad UI](img/QtDesignerNotepad.png)

Once we have a nice design `.ui` file, we need to convert it to a Python `.py` file.

This is done using a tool installed by `PySide` called `pyside-uic`:

In [34]:
!pyside-uic ..\scripts\notepad.ui -o notepad_design.py

In [10]:
%less notepad_design.py

Remember that you need to run this command everytime you change the `.ui` file in the designer.

Now we can import the main window from the `notepad_design.py` and create a new window that inherits from this our design:

In [20]:
from notepad_design import Ui_MainWindow

class MainWindow(QtGui.QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setupUi(self)
        self.show()

window = MainWindow()

Of course, to make this interactive we need to implement button callbacks and other UI logic. The names of the buttons were defined in the designer, but we can find them in the windows object:

In [25]:
list(window.__dict__.keys())

['browseButton',
 'saveButton',
 'statusbar',
 'textEdit',
 'menubar',
 'filenameEdit',
 'centralwidget',
 'filenameLabel',
 'loadButton']

We can also get help on eacho of the widgets in the window or look at the [docs](https://srinikom.github.io/pyside-docs/PySide/QtGui/QTextEdit.html):

In [29]:
help(window.textEdit)

Help on QTextEdit object:

class QTextEdit(QAbstractScrollArea)
 |  Method resolution order:
 |      QTextEdit
 |      QAbstractScrollArea
 |      QFrame
 |      QWidget
 |      PySide.QtCore.QObject
 |      QPaintDevice
 |      Shiboken.Object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __new__(*args, **kwargs) from Shiboken.ObjectType
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  acceptRichText(...)
 |  
 |  alignment(...)
 |  
 |  anchorAt(...)
 |  
 |  append(...)
 |  
 |  autoFormatting(...)
 |  
 |  canInsertFromMimeData(...)
 |  
 |  canPaste(...)
 |  
 |  changeEvent(...)
 |  
 |  clear(...)
 |  
 |  contextMenuEvent(...)
 |  
 |  copy(...)
 |  
 |  createMimeDataFromSelection(...)
 |  
 |  createStandardContextMenu(...)
 |  
 |  currentCharFormat(...)
 |  
 |  currentFont(...)
 |  
 |  cursorForPositio

## Save button

We start with the save button. We need to:

- implement a new method, `save` that 
  - reads the text from the editor
  - reads the filename from the filename textbox
  - opens a file and write the text
  - catches exceptions and reports them with a dialog (like the first example in this session)
- connect the method to the button

In [46]:
from notepad_design import Ui_MainWindow

class MainWindow(QtGui.QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setupUi(self)
        
        self.saveButton.clicked.connect(self.save)
        
        self.show()
        
    def save(self):
        text = self.textEdit.toPlainText()
        filename = self.filenameEdit.text()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            msg = QtGui.QMessageBox(self)
            msg.setText(str(e))
            msg.show()
                

window = MainWindow()

Note that while the window is open and even after you close it, you can introspect it in the notebook:

In [43]:
print(window.filenameEdit.text())

tmp.txt


In [45]:
fname = window.filenameEdit.text()
%less $fname

## Load button

Next, we write a `load` method, which
- reads a filename from the filename textbox
- reads the text from the file
- puts the text in the editor
We then connect method to the load button.

In [62]:
from notepad_design import Ui_MainWindow

class MainWindow(QtGui.QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setupUi(self)
        
        self.saveButton.clicked.connect(self.save)
        self.loadButton.clicked.connect(self.load)
        
        self.show()
    
    def error_dialog(self, error):
        msg = QtGui.QMessageBox(self)
        msg.setText(str(error))
        msg.show()
        
    def save(self):
        text = self.textEdit.toPlainText()
        filename = self.filenameEdit.text()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
                self.error_dialog(e)
    
    def load(self):
        filename = self.filenameEdit.text()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            self.error_dialog(e)
        self.textEdit.clear()
        self.textEdit.append(text)
        
window = MainWindow()

In [63]:
window.textEdit.toPlainText()

'dasdasdasdasds\ndasdasd\n'

## Browser button

Lastly, we will implement the browse button that will open a [file dialog](https://srinikom.github.io/pyside-docs/PySide/QtGui/QFileDialog.html) and save the result to the filename textbox. Let's first experiment:

In [71]:
filename, _ = QtGui.QFileDialog.getOpenFileName(window, "Open Text File", ".", "Text Files (*.txt *.csv *.json)")
print(filename)

D:/workspace/Py4Eng/sessions/tmp.txt


In [77]:
from notepad_design import Ui_MainWindow

class MainWindow(QtGui.QMainWindow, Ui_MainWindow):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setupUi(self)
        
        self.saveButton.clicked.connect(self.save)
        self.loadButton.clicked.connect(self.load)
        self.browseButton.clicked.connect(self.browse)
        
        self.show()
    
    def error_dialog(self, error):
        msg = QtGui.QMessageBox(self)
        msg.setText(str(error))
        msg.show()
        
    def save(self):
        text = self.textEdit.toPlainText()
        filename = self.filenameEdit.text()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
                self.error_dialog(e)
    
    def load(self):
        filename = self.filenameEdit.text()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            self.error_dialog(e)
        self.textEdit.clear()
        self.textEdit.append(text)
        
    def browse(self):
        filename, _ = QtGui.QFileDialog.getOpenFileName(
            self, 
            "Open Text File", 
            ".", 
            "Text Files (*.txt *.csv *.json)"
        )
        self.filenameEdit.setText(filename)    
        
window = MainWindow()

That's it, we have our notepad application.

## Exercise

Add a button called "All Caps" that changes the text to uppercase.

# Qt and Matplotlib

Matplotlib has support for Qt via PySide, so that we can put a plot inside a `QWidget`,

We have some importing to do. We first import `matplotlib` and set it to use the `Qt4Agg`. It's important to do this before any other matplotlib-related import. We then tell it that the Qt4 backend should be PySide (at the time I write this, March 2016, PySide doesn't support Qt5).

We then import from matplotlib a Qt-specialized canvas and navigation toolbar (this is optional, used for zomming and tilting).

We also import NumPy and Seaborn.

In [2]:
%gui qt
from PySide import QtGui
import PySide
print("PySide version: ", PySide.__version__)

import matplotlib
print("Matplotlib version:", matplotlib.__version__)
matplotlib.use('Qt4Agg')
matplotlib.rcParams['backend.qt4']='PySide'

from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt4agg import NavigationToolbar2QT as NavigationToolbar

import numpy as np
import seaborn as sns
sns.set(
    style='white',
    palette='muted'
)

PySide version:  1.2.2
Matplotlib version: 1.5.1


In [13]:
class MainWindow(QtGui.QWidget):
    def __init__(self):
        super().__init__()
        # create a plot widget
        self.fig = matplotlib.figure.Figure(figsize=(6,4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvas(self.fig)
        self.canvas.setParent(self)
                
        self.mpl_toolbar = NavigationToolbar(self.canvas, self)
        
        left_vbox = QtGui.QVBoxLayout()
        left_vbox.addWidget(self.canvas)
        left_vbox.addWidget(self.mpl_toolbar)

        hbox = QtGui.QHBoxLayout()
        hbox.addLayout(left_vbox)
        self.setLayout(hbox)
        
        self.show()

window = MainWindow()

Now we can plot to the window from the notebook. Most of this code is the same as we always do with matplotlt, only that:
- we already have `fig` and `ax` objects, defined in the `__init__` above cell
- when we are done plotting we need to call `window.canvas.draw()` to make it update the GUI.

In [14]:
x = np.linspace(0, 2 * np.pi, 100)
y1 = np.sin(x)
y2 = np.cos(x)

window.ax.plot(x, y1)
window.ax.plot(x, y2)
window.ax.set(
    xlabel='x',
    ylabel='y',
    xlim=(x.min(), x.max()),
    ylim=(y.min(), y.max()),
)
window.fig.tight_layout()
window.canvas.draw()

## Full application

We can incorporate matplotlib in a full application, similar to our notepad from above. 

See for example `trigoplot.py` in the `scripts` folder - run it with `trigoplot.bat` that also runs `pyside-uic` on `trigoplot.ui`.

## Colophon
This notebook was written by [Yoav Ram](http://www.yoavram.com) and is part of the _Python for Engineers_ course.

The notebook was written using [Python](http://pytho.org/) 3.4.4, [IPython](http://ipython.org/) 4.0.3 and [Jupyter](http://jupyter.org) 4.0.6.

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)